In [2]:
# importing dependencies
import pandas as pd
from sqlalchemy import create_engine


In [3]:
#importing imdb files
imdb_ratings_path = "resources/IMDb ratings.csv"
ratings_df = pd.read_csv(imdb_ratings_path)


In [4]:
ratings_df.columns
ratings_min = ratings_df[['imdb_title_id','us_voters_rating', 'non_us_voters_rating','us_voters_votes', 'non_us_voters_votes']]
ratings_min.head()

,imdb_title_id,us_voters_rating,non_us_voters_rating,us_voters_votes,non_us_voters_votes
0,tt0000009,6.4,6.0,51.0,70.0
1,tt0000574,6.0,6.2,96.0,331.0
2,tt0001892,6.2,5.9,31.0,123.0
3,tt0002101,5.5,4.7,207.0,105.0
4,tt0002130,7.0,7.0,488.0,1166.0


In [5]:
col = ratings_min.loc[:,'us_voters_rating':'non_us_voters_rating']

ratings_plus = ratings_min
ratings_plus['average_total_ratings'] = col.mean(axis=1)
ratings_plus.head()

/Users/rchaud/opt/anaconda3/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,imdb_title_id,us_voters_rating,non_us_voters_rating,us_voters_votes,non_us_voters_votes,average_total_ratings
0,tt0000009,6.4,6.0,51.0,70.0,6.20
1,tt0000574,6.0,6.2,96.0,331.0,6.10
2,tt0001892,6.2,5.9,31.0,123.0,6.05
3,tt0002101,5.5,4.7,207.0,105.0,5.10
4,tt0002130,7.0,7.0,488.0,1166.0,7.00


In [6]:
col1 = ratings_min.loc[:,'us_voters_votes':'non_us_voters_votes']

ratings_plus['average_total_votes'] = col1.mean(axis=1)
ratings_plus.head()

/Users/rchaud/opt/anaconda3/envs/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,imdb_title_id,us_voters_rating,non_us_voters_rating,us_voters_votes,non_us_voters_votes,average_total_ratings,average_total_votes
0,tt0000009,6.4,6.0,51.0,70.0,6.20,60.5
1,tt0000574,6.0,6.2,96.0,331.0,6.10,213.5
2,tt0001892,6.2,5.9,31.0,123.0,6.05,77.0
3,tt0002101,5.5,4.7,207.0,105.0,5.10,156.0
4,tt0002130,7.0,7.0,488.0,1166.0,7.00,827.0


In [8]:
connection_string = "postgres:posgres@localhost:5432/movies_db"
engine = create_engine(f'postgresql://{connection_string}')
ratings_plus.to_sql(name='ratings', con=engine, if_exists='replace', index=False)